# Web Scraping the Data

In this section the data is scraped from the Wikipedia page: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M. This is used to create a table that stores the postal code, borough, and neighboors.

In [175]:
import numpy as np
import pandas as pd

import json
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors

from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
from pandas.io.json import json_normalize
import folium

This is grabbing everything from the webpage.

In [47]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
hmtl_data = requests.get(url).text

This is changing everything that was grabbed to be read as html code.

In [48]:
soup = BeautifulSoup(hmtl_data, "html5lib")

This section starts by creating an empty list to store the data and load into the DataFrame t_n. Next the code searches for the a table in the html data code, storing it in table. The for loop will go through each line. The loop will skip if the line has 'Not assigned' as a value. If the line doesn't have that it will change the line to text, remove unneeded characters and store it in the made list. After looping through the whole table the list is stored into a DataFrame. The last line cleans up the names of some of the boroughs stored in the DataFrame.

In [49]:
table_contents=[]
#t_n = pd.DataFrame(columns=['PostalCode', 'Borough', 'Neighborhoods']
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
t_n=pd.DataFrame(table_contents)
t_n['Borough']=t_n['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

This is showing the newly created DataFrame.

In [50]:
t_n

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


And this is the shape of the Dataframe.

In [51]:
t_n.shape

(103, 3)

# Latitude and Longitude

This section grabs the Latitude and Longitude from the Geospatial_Coordinates.csv and stores it in a DataFrame. From there it merges the new DataFrame and the one from the previous section.

The CSV file, Geospatial_Coordinates.csv, is store into a DataFrame. The columns are renamed to be used in the next section. The newly form DataFrame is displayed.

In [ ]:
la_lng = pd.read_csv("Geospatial_Coordinates.csv")
names = ['PostalCode', 'Latitude', 'Longitude']
la_lng.columns = names
la_lng

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


This merges the two Dataframe on the column "PostalCode".

In [122]:
t_neghborhood = pd.merge(t_n, la_lng, on="PostalCode")
t_neghborhood

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


# Clustering the Boroughs

The Boroughs are grouped together and averaging their Latitude and Longitude.

In [123]:
t_neghborhood = t_neghborhood.drop(['PostalCode', 'Neighborhood'], axis=1)
Borough = t_neghborhood.groupby('Borough').mean().reset_index()

Displaying the new DataFrame.

In [124]:
Borough

,Borough,Latitude,Longitude
0,Central Toronto,43.701980,-79.398954
1,Downtown Toronto,43.654624,-79.384184
2,Downtown Toronto Stn A,43.646435,-79.374846
3,East Toronto,43.671110,-79.325428
4,East Toronto Business,43.662744,-79.321558
5,East York,43.704043,-79.335287
6,East York/East Toronto,43.685347,-79.338106
7,Etobicoke,43.655797,-79.537348
8,Etobicoke Northwest,43.706748,-79.594054
9,Mississauga,43.636966,-79.615819


Setting the information used for getting the data from FourSquare.

In [180]:
CLIENT_ID = '' 
CLIENT_SECRET = ''
VERSION = '20180605' # Foursquare API version
LIMIT = 100 
radius = 500

Defining a function to grab the Venues, their locations, category, and  place them into a DataFrame.

In [133]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Getting the data for the Borough of Toronto.

In [136]:
T_data = getNearbyVenues(names=Borough['Borough'], latitudes=Borough['Latitude'], longitudes=Borough['Longitude'])
T_data

,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Central Toronto,43.701980,-79.398954,Balsamico,43.701505,-79.397162,Italian Restaurant
1,Central Toronto,43.701980,-79.398954,Tabülè,43.700932,-79.397078,Middle Eastern Restaurant
2,Central Toronto,43.701980,-79.398954,Little Sister,43.701552,-79.397163,Indonesian Restaurant
3,Central Toronto,43.701980,-79.398954,La Salumeria,43.701255,-79.397117,Deli / Bodega
4,Central Toronto,43.701980,-79.398954,Five Doors North,43.702236,-79.397526,Italian Restaurant
...,...,...,...,...,...,...,...
338,West Toronto,43.652653,-79.449290,Mister Yummy,43.656793,-79.451724,Korean Restaurant
339,York,43.690797,-79.472633,Caldense Bakery,43.691993,-79.467512,Dessert Shop
340,York,43.690797,-79.472633,Subway,43.690218,-79.474050,Sandwich Place
341,York,43.690797,-79.472633,Dollar Tree,43.690296,-79.474667,Discount Store


Counting the Venues per Borough.

In [137]:
T_data.groupby('Borough').count()

,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Borough,,,,,,
Central Toronto,29,29,29,29,29,29
Downtown Toronto,67,67,67,67,67,67
Downtown Toronto Stn A,99,99,99,99,99,99
East Toronto,25,25,25,25,25,25
East Toronto Business,17,17,17,17,17,17
East York,5,5,5,5,5,5
East York/East Toronto,4,4,4,4,4,4
Etobicoke,2,2,2,2,2,2
Etobicoke Northwest,5,5,5,5,5,5


Checking the amount of different Venue Category.

In [145]:
print('There are {} uniques categories.'.format(len(T_data['Venue Category'].unique())))

There are 137 uniques categories.
<class 'pandas.core.series.Series'>


Creating a onehot encoder for all the unique categories and creating a DataFrame.

In [148]:
Borough_onehot = pd.get_dummies(T_data[['Venue Category']], prefix="", prefix_sep="")
Borough_onehot['Borough'] = T_data['Borough']
fixed_columns = [Borough_onehot.columns[-1]] + list(Borough_onehot.columns[:-1])
Borough_onehot = Borough_onehot[fixed_columns]
Borough_onehot.head()

,Borough,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Entertainment,Asian Restaurant,Auto Workshop,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Belgian Restaurant,Bistro,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burrito Place,Business Service,Café,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Cafeteria,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,...,Pakistani Restaurant,Park,Pharmacy,Pizza Place,Plaza,Poke Place,Pub,Ramen Restaurant,Recording Studio,Rental Car Location,Restaurant,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Sporting Goods Shop,Sushi Restaurant,Syrian Restaurant,Tailor Shop,Tanning Salon,Thai Restaurant,Theater,Tibetan Restaurant,Trail,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Central Toronto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Central Toronto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Central Toronto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Central Toronto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Grouping the hot encoder by Borough.

In [151]:
Borough_g = Borough_onehot.groupby('Borough').mean().reset_index()
Borough_g

,Borough,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Entertainment,Asian Restaurant,Auto Workshop,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Belgian Restaurant,Bistro,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burrito Place,Business Service,Café,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Cafeteria,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,...,Pakistani Restaurant,Park,Pharmacy,Pizza Place,Plaza,Poke Place,Pub,Ramen Restaurant,Recording Studio,Rental Car Location,Restaurant,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Sporting Goods Shop,Sushi Restaurant,Syrian Restaurant,Tailor Shop,Tanning Salon,Thai Restaurant,Theater,Tibetan Restaurant,Trail,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.034483,0.034483,0.034483,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.034483,0.000000,0.000000,0.034483,0.000000,0.000000,0.000000,0.000000,0.000000,0.068966,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.034483,0.000000,0.000000,0.000000,...,0.00,0.000000,0.034483,0.034483,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.068966,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.0,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.137931,0.034483,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
1,Downtown Toronto,0.000000,0.000000,0.014925,0.014925,0.000000,0.00,0.000000,0.000000,0.000000,0.014925,0.000000,0.000000,0.000000,0.000000,0.000000,0.029851,0.000000,0.000000,0.029851,0.000000,0.000000,0.000000,0.014925,0.000000,0.014925,0.000000,0.104478,0.014925,0.104478,0.000000,0.000000,0.000000,0.014925,0.014925,0.0,0.000000,0.029851,0.000000,0.000000,...,0.00,0.000000,0.000000,0.014925,0.029851,0.014925,0.014925,0.014925,0.000000,0.0,0.014925,0.014925,0.014925,0.014925,0.014925,0.00,0.000000,0.00,0.0,0.0,0.000000,0.000000,0.00,0.000000,0.014925,0.014925,0.000000,0.000000,0.014925,0.014925,0.029851,0.00,0.000000,0.0,0.014925,0.014925,0.014925,0.000000,0.0,0.000000
2,Downtown Toronto Stn A,0.000000,0.010101,0.020202,0.000000,0.000000,0.00,0.000000,0.010101,0.030303,0.000000,0.000000,0.010101,0.030303,0.010101,0.010101,0.010101,0.020202,0.000000,0.000000,0.000000,0.000000,0.000000,0.030303,0.020202,0.000000,0.010101,0.010101,0.030303,0.121212,0.000000,0.000000,0.010101,0.000000,0.010101,0.0,0.000000,0.010101,0.020202,0.000000,...,0.00,0.020202,0.020202,0.000000,0.000000,0.010101,0.020202,0.000000,0.000000,0.0,0.030303,0.010101,0.040404,0.000000,0.000000,0.00,0.000000,0.00,0.0,0.0,0.000000,0.000000,0.00,0.010101,0.010101,0.010101,0.000000,0.010101,0.000000,0.010101,0.000000,0.00,0.000000,0.0,0.010101,0.000000,0.000000,0.000000,0.0,0.010101
3,East Toronto,0.000000,0.000000,0.040000,0.000000,0.000000,0.04,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.080000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.040000,0.000000,0.000000,0.000000,...,0.04,0.040000,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.04,0.000000,0.04,0.0,0.0,0.000000,0.000000,0.04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.04,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
4,East Toronto Business,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.00000

Creating a function that returns a index of the most common Venue Category.

In [156]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Creating a DataFrame with the most common Venue Category.

In [172]:
num_ten = 10
indicators = ['st', 'nd', 'rd']

columns = ['Borough']
for ind in np.arange(num_ten):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

Borough_venues_sorted = pd.DataFrame(columns=columns)
Borough_venues_sorted['Borough'] = Borough_g['Borough']

for ind in np.arange(Borough_g.shape[0]):
    Borough_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Borough_g.iloc[ind, :], num_ten)

Borough_venues_sorted.head()

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Italian Restaurant,Sushi Restaurant,Coffee Shop,Restaurant,Pharmacy,Buffet,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Metro Station
1,Downtown Toronto,Clothing Store,Coffee Shop,Theater,Diner,Bookstore,Cosmetics Shop,Middle Eastern Restaurant,Bubble Tea Shop,Plaza,Neighborhood
2,Downtown Toronto Stn A,Coffee Shop,Seafood Restaurant,Hotel,Cocktail Bar,Japanese Restaurant,Italian Restaurant,Bakery,Beer Bar,Restaurant,Café
3,East Toronto,Indian Restaurant,Café,Grocery Store,Bar,Shopping Plaza,Skating Rink,Convenience Store,Brewery,Snack Place,Pizza Place
4,East Toronto Business,Yoga Studio,Brewery,Gym / Fitness Center,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Light Rail Station,Comic Shop,Park


Creating the Clustering using Kmeans. The model will run 15 times using the hot encoder data.

In [169]:
B_data = Borough_g.drop('Borough', axis=1)
kmeans = KMeans(n_clusters=4, init='k-means++', n_init=15, random_state=0).fit(B_data)
print(kmeans.labels_[:])

[0 0 0 0 0 2 2 1 0 0 3 0 0 0 0]


Creating a new DataFrame with the Borough, Latitude, Lonitude, Cluster Value, and the ten most Common Venue Category.

In [173]:
Borough_venues_sorted.insert(0, 'Cluster Values', kmeans.labels_)
Final_data = pd.merge(Borough, Borough_venues_sorted)
Final_data.head()

,Borough,Latitude,Longitude,Cluster Values,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,43.701980,-79.398954,0,Italian Restaurant,Sushi Restaurant,Coffee Shop,Restaurant,Pharmacy,Buffet,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Metro Station
1,Downtown Toronto,43.654624,-79.384184,0,Clothing Store,Coffee Shop,Theater,Diner,Bookstore,Cosmetics Shop,Middle Eastern Restaurant,Bubble Tea Shop,Plaza,Neighborhood
2,Downtown Toronto Stn A,43.646435,-79.374846,0,Coffee Shop,Seafood Restaurant,Hotel,Cocktail Bar,Japanese Restaurant,Italian Restaurant,Bakery,Beer Bar,Restaurant,Café
3,East Toronto,43.671110,-79.325428,0,Indian Restaurant,Café,Grocery Store,Bar,Shopping Plaza,Skating Rink,Convenience Store,Brewery,Snack Place,Pizza Place
4,East Toronto Business,43.662744,-79.321558,0,Yoga Studio,Brewery,Gym / Fitness Center,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Light Rail Station,Comic Shop,Park


Creating a map with the clustering.

In [179]:
map_clusters = folium.Map(location=[43.6532, -79.3832], zoom_start=11)

x = np.arange(4)
ys = [i + x + (i*x)**2 for i in range(4)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(Final_data['Latitude'], Final_data['Longitude'], Final_data['Borough'], Final_data['Cluster Values']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters